# Anomaly Detection Data Preprocessing
## 0. Prerequisites
### Operating System
If you are using Mac OS, Linux, or WSL on Windows, skip to **Python Package Dependencies**.

Otherwise, follow the steps in [Anaconda_setup.ipynb](https://github.com/451488975/Anaconda_Setup) to set up WSL on Windows. Since WSL currently does not have GPU support, follow the steps in the tutorial labelled with "w/o NVIDIA GPU."
### Python Package Dependencies
Install the following dependencies:
* numpy
* matplotlib
* pandas
* tqdm
* h5py
* pyjet (not supported on Windows)

If you have Anaconda, first try installing them via Anaconda (make sure you're in the right environment!) by running `conda install [package name]` in terminal. If the package is not found by Anaconda, install with pip by running `pip install [package name]` in terminal.
### Data
We will be working with the [LHCO2020 R&D Dataset](https://zenodo.org/record/3832254#.X9VHi9hKguU). Download `events_anomalydetection.h5` from the link.


In [1]:
# Load all our modules

import h5py as h5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pyjet import cluster, DTYPE_PTEPM
from pyjet.testdata import get_event

ModuleNotFoundError: No module named 'pyjet'

## 1. Loading Data
### Reading Data
Our data is stored as pandas dataframes in the compressed `.h5` file format. Since it's a large dataset, we can't read it in its entirety because it will likely exceed our RAM capacity. For the purposes of feature study, we do not need to read the entire dataset.

In [3]:
# Replace this variable with your relative path to datafile
datapath = "data/events_anomalydetection.h5"

df = pd.read_hdf(datapath, stop = 100000) # The amount of data loaded can be reduced by an order of magnitude on older hardware
print("Shape: " + str(df.shape))
print("Memory usage in GB:", round(sum(df.memory_usage(deep=True)) / (1024**3), 3))

Shape: (100000, 2101)
Memory in GB: 1.5661120414733887


### Understanding Data
The data is described on the [Zenodo site](https://zenodo.org/record/3832254#.X9VHi9hKguU). In essence, each row corresponds to an event, which is either signal (W'->X+Y) or background QCD dijet.

The shape of the original dataset is `(1100000,2101)`. There are 1M background rows and 100k signal rows. The last column contains a single bit that indicates (for the purpose of R&D) `0` if the row is background and `1` if the row is signal.

Each row consists of up to 700 massless final-state, charged, hadronic particles recorded in `(pT,eta,phi)` triplets. Most rows don't have 700 non-trivial particles, and have instead been zero-padded up to 700 triplets.

The signal event consists of a hypothetical hadron W' with rest mass 3.5 TeV decaying into hypothetical hadrons X and Y with rest masses 500 GeV and 100 GeV, respectively. We expect the W' to be non-relativistic in the lab frame. We expect X to have total energy 1785 GeV and Y to have total energy 1716 GeV. Optionally, you can view the [derivation](https://www.overleaf.com/read/mpmgzzdcsyxz
).

X and Y then decay into two quarks each: X->qq, Y->qq. Since X and Y are relativistic, the quarks will be part of the same jet; that is, the signal event consists of two jets, one associated with X and one associated with Y. Each one of those jets will have two-pronged substructure, which we can see later by studying the tau2/tau1 feature.

In [ ]:
# Observe the following about the data:
# pT columns are always positive
# Zero-padding is evident at the end
# Last column (index 2100) is the signal bit
# Signal and background are randomly shuffled within the data
df

In [ ]:
# Split the data based on the signal bit and discard the flag column
df_bg = df[df[2100] == 0.0].iloc[:,:-1]
df_s = df[df[2100] == 1.0].iloc[:,:-1]
df = df.iloc[:,:-1]

In [ ]:
print("Background shape: " + str(df_bg.shape))
print("Signal shape: " + str(df_s.shape))
print("Percent of signal in data: " + str(round(df_s.shape[0] / df.shape[0] * 100, 2)) + "%")

## 2. Cluster Jets
The goal is to cluster each row into two jets to reflect the W'->X,Y structure of the signal.

## 3. Plot Features

## 4. Exercise
Now it's your turn! Do the data preprocessing and feature study, as we've done in this tutorial, using the file `events_anomalydetection_Z_XY_qqq.h5`, which you should download at the same [Zenodo link](https://zenodo.org/record/3832254#.X9VNZdhKguU). Note a few key differences:
* This file only has 100k signal events and no background
* Therefore, there is no signal bit column
* The signal has 3-prong substructure, so the proper N-subjetiness ratio to compute is tau3/tau2 instead of tau2/tau1